<a href="https://colab.research.google.com/github/isabelacfs/home/blob/master/clickstream_isabela_30112022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Entendendo interesses dos clientes através de uma análise dos cliques do usuário em uma plataforma de compras on-line





Em uma loja de rua, não é preciso muito treinamento para que os atendentes percebam quais são os comportamentos típicos do cliente que vai levar ou não algum produto. 
Mas em uma loja on-line a coisa é diferente! As interações dentro da loja são detectadas através de outra linguagem: fluxos de cliques(clickstream) e movimentos com o mouse podem nos dar dicas sobre o comportamento de consumo.

Este noteook é a primeira etapa desta análise e visa preparar os dados para que possamos entender melhor os interesses dos clientes através da sua interação. Essa informação nos permite perceber tendências e até mesmo planejar de forma mais assertiva as campanhas de marketing, possibilitando uma melhor receptividade da oferta e, em última análise, melhora do retorno sobre o valor investido em marketing.



### Inspiração

Este notebook foi inspirado em um projeto publicado orginalmente pelo time da IBM. 
O arquivo extraído do notebook original econtra-se em CSV e contém eventos que representam as diversas interações dos usuários em uma plataforma de compras fictícia, desde a **visualização das páginas, adicionar ao carrinho e realização de pedido**. 
</br>
Utilizamos esse arquivo CSV e realizamos diversas consultas, limpeza e calculamos as métricas agregadas que, no projeto original, foram feitas em Spark. Aqui forçamos a utilização do Python para efeitos de auto-aprendizado.
A ideia foi **replicar o racionínio e as descobertas feitas na análise e extração dessas métricas**. 



</br> Neste notebook realizamos a **limpeza, preparo, exploração, extração de métricas agregadas**. E, por fim, exportamos o arquivo em excel para visualização de dados no Tableau (projeto a ser publicado na sequência). 

Embora o conjunto de dado tenha sofrido pequenas atualizações desde a última publicação pelo time IBM, o arquivo que utilizei ainda preserva praticamente as mesmas conclusões. 

</br> Link para o projeto original

O dataset original encontra-se em https://raw.githubusercontent.com/IBM/db2-event-store-clickstream/master/data/clickstream_data.csv 

Github:  onde os dados são extraídos de um banco de dados DB2 originalmente no endereço: </br> https://github.com/IBM/db2-event-store-clickstream/blob/master/README.md

#### Carregando dados e importando pacotes necessários

In [ ]:
from numpy import int64
import pandas as pd
from datetime import datetime
PATH = 'https://raw.githubusercontent.com/IBM/db2-event-store-clickstream/master/data/clickstream_data.csv'
df = pd.read_csv(PATH, dtype={'timestamp': int})

#### Preparando os dados

In [ ]:
# renomeando a coluna de nome desconhecido
df.rename(columns={'Unnamed: 8': 'nomeDesconhecido'}, inplace=True)

Mostrando as primeiras linhas

In [ ]:
df.head(5)

,eventId,eventType,timestamp,ipaddress,sessionId,userId,pageUrl,browser,nomeDesconhecido,timestampToDate,time,product_line,product,action,feature,shopping_cart
0,20170522901,pageView,1496311260,169.34.56.78,y20170522a4499u21,ceaton,www.cybershop.com,Chrome,1,2017-06-01 10:01:00,0.0,,,,,
1,20170522902,pageView,1496311320,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&action=catalog,Chrome,2,2017-06-01 10:02:00,60.0,smartphone,,catalog,,
2,20170522903,pageView,1496311440,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&action=catalog,Chrome,1,2017-06-01 10:04:00,120.0,smartphone,,catalog,,
3,20170522904,pageView,1496311500,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&product=A-phon...,Chrome,1,2017-06-01 10:05:00,60.0,smartphone,A-phone,details,,
4,20170522905,pageView,1496311560,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&product=A-phon...,Chrome,1,2017-06-01 10:06:00,60.0,smartphone,A-phone,details,color,


Sobre o dataset

In [ ]:
# tamanho do dataset
print('O dataset tem {0} linhas e {1} colunas'.format(df.shape[0], df.shape[1]))

O dataset tem 123 linhas e 9 colunas


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   eventId           123 non-null    int64 
 1   eventType         123 non-null    object
 2   timestamp         123 non-null    int64 
 3   ipaddress         123 non-null    object
 4   sessionId         123 non-null    object
 5   userId            123 non-null    object
 6   pageUrl           123 non-null    object
 7   browser           123 non-null    object
 8   nomeDesconhecido  123 non-null    int64 
dtypes: int64(3), object(6)
memory usage: 8.8+ KB


#### Parâmetros page Url e timestamp


Veja que o parâmetro page URL contém informações importantes sobre o endereço visitado, ações tomadas pelo usuário, produtos e linhas de produtos os quais ele se interessou em determinado momento.

A coluna timestamp representa o instante específico em que a ação ou visita aconteceu. A unidade utilizada está no padrão Unix, ou seja, a **quantidade de segundos desde o dia primeiro de janeiro de 1970 até a ocorrência do fato**. 

#### Preparando page URL

Encontramos uma / (barra) no link /www.cybershop.com e vamos retirar para que não fique diferente da outra. 

In [ ]:
df.loc[df['pageUrl'].str.contains(r'/www.cybershop.com')].shape

(0, 16)

In [ ]:
# encontrando todas
print('Existem {0} ocorrências de / na frente da URL cybershop'.format(df.loc[df['pageUrl'].str.contains(r'/www.cybershop.com')].shape[0]))

Existem 0 ocorrências de / na frente da URL cybershop


In [ ]:
# retirando as barras que aparecerem como caractere inicial
df['pageUrl'] = df['pageUrl'].str.strip('/')
df['pageUrl']

0                                      www.cybershop.com
1         estore?product_line=smartphones&action=catalog
2         estore?product_line=smartphones&action=catalog
3      estore?product_line=smartphones&product=A-phon...
4      estore?product_line=smartphones&product=A-phon...
                             ...                        
118    estore?product_line=smartphones&product=A-phon...
119    estore?product_line=smartphones&product=X-phon...
120    estore?product_line=smartphones&product=A-phon...
121    estore?product_line=smartphones&product=X-phon...
122                                    www.cybershop.com
Name: pageUrl, Length: 123, dtype: object

### Converter Timestamp para data


Para esta conversão, primeiro, mudamos a coluna criamos uma **nova coluna chamado timestamp to date** e então convertemos a original para inteiro e, em seguida para string. Desta forma a conversão para datetime funcionou.



In [ ]:
df['timestampToDate'] = df['timestamp'].astype(int).astype(str)
df.dtypes

eventId              int64
eventType           object
timestamp            int64
ipaddress           object
sessionId           object
userId              object
pageUrl             object
browser             object
nomeDesconhecido     int64
timestampToDate     object
dtype: object

Documentação do pandas sobre datetime
https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html?highlight=to_datetime#pandas.to_datetime



</br>

A documentação nos diz que quando encontramos um time stamp com 10 dígitos, estamos diante de um formato que está na unidade em segundos. 


```
pd.to_datetime(1490195805, unit='s')

pd.to_datetime(1490195805433502912, unit='ns')
```




In [ ]:
# criando a nova coluna de data a partir da coluna timestamp
df['timestampToDate'] = pd.to_datetime(df['timestampToDate'], origin='unix', unit='s')
df['timestampToDate']

0     2017-06-01 10:01:00
1     2017-06-01 10:02:00
2     2017-06-01 10:04:00
3     2017-06-01 10:05:00
4     2017-06-01 10:06:00
              ...        
118   2017-06-22 14:09:00
119   2017-06-22 14:10:00
120   2017-06-22 14:22:00
121   2017-06-22 14:23:00
122   2017-06-22 14:27:00
Name: timestampToDate, Length: 123, dtype: datetime64[ns]

## Extraindo as Principais Métricas



#### Calculando o tempo gasto nas páginas

Calculamos a diferença de tempo entre o clique(ou ação)atual pelo anterior.

In [ ]:
# diferença
delta = df['timestamp'] - df['timestamp'].shift()
# cria coluna time e atribui esta diferença
df['time'] = delta

In [ ]:
df['time']

0        NaN
1       60.0
2      120.0
3       60.0
4       60.0
       ...  
118     60.0
119     60.0
120    720.0
121     60.0
122    240.0
Name: time, Length: 123, dtype: float64

E depois preenchemos valores que sobraram vazios com zero

In [ ]:
# Preenche o valor NaN com zero
df['time'].fillna(0, inplace=True)

In [ ]:
df['time']

0        0.0
1       60.0
2      120.0
3       60.0
4       60.0
       ...  
118     60.0
119     60.0
120    720.0
121     60.0
122    240.0
Name: time, Length: 123, dtype: float64

#### Page hits

 Calcula page hits

Primeiro, cria um dataframe separado com a quantidade de page hits por url visitada.

In [ ]:
# cria um df com a quantidade de page hits por url visitada
dfCountHits = pd.DataFrame(df.loc[df.eventType == 'pageView', 'pageUrl'].value_counts())
dfCountHits.reset_index(inplace=True)
dfCountHits.rename(columns={'index': 'pageUrl', 'pageUrl': 'pageHits'}, inplace=True)

Criar df agrupado por url visitada utilizando também as demais métricas agregadas como colunas.

In [ ]:
# criar df agrupado por url visitada e as métricas agregadas
dfHits = pd.DataFrame(df.loc[df.eventType == 'pageView'].groupby('pageUrl').sum())
dfHits.reset_index(inplace=True)


#### Mesclar (Inner ou merge)

Interseção entre o df com a métrica de hits por url e o df com as demais métricas agregadas

In [ ]:
dfPageHits = pd.merge(dfHits, dfCountHits, on='pageUrl')


In [ ]:
dfHits.shape
dfCountHits.shape

(46, 2)

In [ ]:
dfPageHits.sort_values(by='pageHits', ascending=False)

,pageUrl,eventId,timestamp,nomeDesconhecido,time,pageHits
21,estore?product_line=smartphones&action=catalog,262216798076,19452102120,21,1260.0,13
45,www.cybershop.com,201705229574,14971281540,10,1810620.0,10
8,estore?product_line=computers&action=catalog,141193660847,10483320060,23,1320.0,7
22,estore?product_line=smartphones&product=A-phon...,141193660632,10477718400,10,480.0,7
43,estore?product_line=videogames&action=catalog,121023137894,8986992960,78,4320.0,6
36,estore?product_line=smartphones&product=S-phon...,100852614754,7486054740,11,660.0,5
25,estore?product_line=smartphones&product=A-phon...,100852614671,7481581380,11,420.0,5
38,estore?product_line=smartphones&product=S-phon...,80682091782,5988331980,9,480.0,4
34,estore?product_line=smartphones&product=M-phon...,80682091898,5990163000,7,480.0,4
24,estore?product_line=smartphones&product=A-phon...,60511569057,4494422280,3,840.0,3


Donwload de Page Hits

In [ ]:
#dfPageHits.to_excel('clickstreamPageHits.xlsx')

#### PageView

Selecionando apenas aqueles eventos que são Page View 

Seleciona apenas os eventos de page view e soma o tempo gasto em cada página

In [ ]:
# "select pageURL, count(*) as page_hits, sum(time) as total_time " +
#   "from ClickData where eventType='pageView' group by pageURL")
dfPageViewWithSeconds = df[df['eventType'] == 'pageView'].groupby(by=['pageUrl']).sum().sort_values(by='time', ascending=False)
dfPageViewWithSeconds


,eventId,timestamp,nomeDesconhecido,time
pageUrl,,,,
www.cybershop.com,201705229574,14971281540,10,1810620.0
estore?product_line=videogames&action=catalog,121023137894,8986992960,78,4320.0
estore?product_line=computers&action=catalog,141193660847,10483320060,23,1320.0
estore?product_line=smartphones&action=catalog,262216798076,19452102120,21,1260.0
estore?product_line=appliances&product=S-washer&action=details,20170522959,1496321220,10,900.0
estore?product_line=smartphones&product=A-phone&feature=camera&action=details,60511569057,4494422280,3,840.0
estore?product_line=smartphones&product=S-phone&action=details,100852614754,7486054740,11,660.0
estore?product_line=computers&product=L-top&action=details,60511568928,4492180980,6,540.0
estore?product_line=smartphones&product=S-phone&feature=camera&action=details,80682091782,5988331980,9,480.0


#### AddToCart e Order

</br>

Vamos ver quanto tempo foi gasto nas páginas considerando os demais eventos que o usuário pode interagir nas páginas como adicionar ao carrinho e pedido.

In [ ]:
# "select pageURL, count(*) as page_hits, sum(time) as total_time " +
#   "from ClickData where eventType='pageView' group by pageURL")
dfOtherEventsWithSeconds = df[df['eventType'] != 'pageView'].groupby(by=['eventType']).sum().sort_values(by='time', ascending=False)
dfOtherEventsWithSeconds



,eventId,timestamp,nomeDesconhecido,time
eventType,,,,
addToCart,221875752439,16465777680,22,2460.0
order,80682091767,5986586280,4,720.0


#### Resumo do Tempo gasto  por página

In [ ]:
df[['eventType', 'time']].groupby(by=['eventType']).sum().sort_values(by='time', ascending=False)


,time
eventType,
pageView,1827180.0
addToCart,2460.0
order,720.0


#### Browser

Tempo gasto por navegador

In [ ]:
df[['time', 'browser']].groupby(by=['browser']).sum().sort_values(by='time', ascending=False)


,time
browser,
Safari,1755420.0
Chrome,69600.0
Firefox,5340.0


 Eventos por navegador

In [ ]:
df[['browser', 'eventType', 'time']].groupby(by=['browser', 'eventType']).sum().sort_values(by='time', ascending=False)


time
browser eventType           
Safari  pageView   1754580.0
Chrome  pageView     68400.0
Firefox pageView      4200.0
        addToCart     1020.0
Chrome  addToCart      840.0
Safari  addToCart      600.0
Chrome  order          360.0
Safari  order          240.0
Firefox order          120.0

#### Usuário

In [ ]:
df[['userId', 'time']].groupby(by=['userId']).sum().sort_values(by='time', ascending=False)

,time
userId,
datkins,1747140.0
sanne,72180.0
cwang,4200.0
mtwain,3000.0
tchoo,1260.0
mking,1200.0
ceaton,1140.0
sraman,240.0


## Extraindo parâmetros de page URL
Extraímos os parâmetros de page url e armazenamos em colunas separadas para cada parâmetro, quando houver.

</br>  Desta forma podemos contabilizar quais e quantas ações cada usuário realizou e o interesse manifestado por cada produto ou linha de produto. 

In [ ]:
# conta quantas ocorrências distintas acontecem para cada endereço acessado
df['pageUrl'].value_counts()

estore?product_line=smartphones&action=catalog                                            13
www.cybershop.com                                                                         10
estore?product_line=smartphones&product=A-phone&action=details                             7
estore?product_line=computers&action=catalog                                               7
estore?product_line=videogames&action=catalog                                              6
estore?product_line=smartphones&product=A-phone&feature=color&action=details               5
estore?product_line=smartphones&product=S-phone&action=details                             5
estore?product_line=smartphones&product=A-phone&action=addToCart                           4
estore?product_line=smartphones&product=S-phone&feature=camera&action=details              4
estore?product_line=smartphones&product=M-phone&action=details                             4
estore?product_line=smartphones&product=A-phone&feature=camera&action=

#### Product line
Cria a coluna de linha de produto

In [ ]:
df['product_line'] = ''

Adiciona os nomes das respectivas linhas de produtos

In [ ]:
df.loc[df['pageUrl'].str.contains('smartphones'), 'product_line'] = 'smartphone'
df.loc[df['pageUrl'].str.contains('computers'), 'product_line'] = 'computers'
df.loc[df['pageUrl'].str.contains('videogames'), 'product_line'] = 'videogames'
df.loc[df['pageUrl'].str.contains('headphones'), 'product_line'] = 'headphones'
df.loc[df['pageUrl'].str.contains('appliances'), 'product_line'] = 'appliances'
df.loc[df['pageUrl'].str.contains('hometheater'), 'product_line'] = 'hometheater'
df.loc[df['pageUrl'].str.contains('cameras'), 'product_line'] = 'cameras'
df.loc[df['pageUrl'].str.contains('carelectronics'), 'product_line'] = 'carelectronics'

#### Product
Cria coluna product e atribui o nome do produto àqueles que contiverem os nomes de produtos correspondentes na URL truncada em page URL

In [ ]:
df['product'] = ''
#smartphones
df.loc[df['pageUrl'].str.contains('A-phone'), 'product'] = 'A-phone'
df.loc[df['pageUrl'].str.contains('S-phone'), 'product'] = 'S-phone'
df.loc[df['pageUrl'].str.contains('X-phone'), 'product'] = 'X-phone'
df.loc[df['pageUrl'].str.contains('M-phone'), 'product'] = 'M-phone'
df.loc[df['pageUrl'].str.contains('L-phone'), 'product'] = 'L-phone'
df.loc[df['pageUrl'].str.contains('H-phone'), 'product'] = 'H-phone'
#other
df.loc[df['pageUrl'].str.contains('M-top'), 'product'] = 'M-top'
df.loc[df['pageUrl'].str.contains('G-washer'), 'product'] = 'G-washer'
df.loc[df['pageUrl'].str.contains('T-lenses'), 'product'] = 'T-lenses'
df.loc[df['pageUrl'].str.contains('Y-receiver'), 'product'] = 'Y-receiver'
df.loc[df['pageUrl'].str.contains('W-game'), 'product'] = 'W-game'
df.loc[df['pageUrl'].str.contains('L-top'), 'product'] = 'L-top'
df.loc[df['pageUrl'].str.contains('S-dryer'), 'product'] = 'S-dryer'
df.loc[df['pageUrl'].str.contains('S-washer'), 'product'] = 'S-washer'
df.loc[df['pageUrl'].str.contains('A-head'), 'product'] = 'A-head'
df.loc[df['pageUrl'].str.contains('B-head'), 'product'] = 'B-head'
df.loc[df['pageUrl'].str.contains('H-top'), 'product'] = 'H-top'
df.loc[df['pageUrl'].str.contains('D-top'), 'product'] = 'D-top'
df.loc[df['pageUrl'].str.contains('D-head'), 'product'] = 'D-head'
df.loc[df['pageUrl'].str.contains('B-theater'), 'product'] = 'B-theater'
df.loc[df['pageUrl'].str.contains('ear-phone'), 'product'] = 'ear-phone'



#### Action

In [ ]:
df['action'] = ''

df.loc[df['pageUrl'].str.contains('catalog'), 'action'] = 'catalog'
df.loc[df['pageUrl'].str.contains('details'), 'action'] = 'details'
df.loc[df['pageUrl'].str.contains('addToCart'), 'action'] = 'addToCart'
df.loc[df['pageUrl'].str.contains('order'), 'action'] = 'order'

#### Feature


In [ ]:
df['feature'] = ''
df.loc[df['pageUrl'].str.contains('color'), 'feature'] = 'color'
df.loc[df['pageUrl'].str.contains('camera'), 'feature'] = 'camera'
df.loc[df['pageUrl'].str.contains('battery'), 'feature'] = 'battery'
df.loc[df['pageUrl'].str.contains('processor'), 'feature'] = 'processor'
df.loc[df['pageUrl'].str.contains('voice_assistant'), 'feature'] = 'voice_assistant'

#### Shopping cart

In [ ]:
df['shopping_cart'] = ''
df.loc[df['pageUrl'].str.contains('c20170622x111'), 'shopping_cart'] = 'c20170622x111'
df.loc[df['pageUrl'].str.contains('c20170622x122'), 'shopping_cart'] = 'c20170622x122'
df.loc[df['pageUrl'].str.contains('c20170622x422'), 'shopping_cart'] = 'c20170622x422'


In [ ]:
df

,eventId,eventType,timestamp,ipaddress,sessionId,userId,pageUrl,browser,nomeDesconhecido,timestampToDate,time,product_line,product,action,feature,shopping_cart
0,20170522901,pageView,1496311260,169.34.56.78,y20170522a4499u21,ceaton,www.cybershop.com,Chrome,1,2017-06-01 10:01:00,0.0,,,,,
1,20170522902,pageView,1496311320,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&action=catalog,Chrome,2,2017-06-01 10:02:00,60.0,smartphone,,catalog,,
2,20170522903,pageView,1496311440,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&action=catalog,Chrome,1,2017-06-01 10:04:00,120.0,smartphone,,catalog,,
3,20170522904,pageView,1496311500,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&product=A-phon...,Chrome,1,2017-06-01 10:05:00,60.0,smartphone,A-phone,details,,
4,20170522905,pageView,1496311560,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&product=A-phon...,Chrome,1,2017-06-01 10:06:00,60.0,smartphone,A-phone,details,color,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,20170523019,pageView,1498140540,119.14.137.99,y20170530a5463u14,sanne,estore?product_line=smartphones&product=A-phon...,Safari,1,2017-06-22 14:09:00,60.0,smartphone,A-phone,details,camera,
119,20170523020,pageView,1498140600,119.14.137.99,y20170530a5463u14,sanne,estore?product_line=smartphones&product=X-phon...,Safari,12,2017-06-22 14:10:00,60.0,smartphone,,details,,
120,20170523021,pageView,1498141320,119.14.137.99,y20170530a5463u14,sanne,estore?product_line=smartphones&product=A-phon...,Safari,1,2017-06-22 14:22:00,720.0,smartphone,A-phone,details,camera,
121,20170523022,pageView,1498141380,119.14.137.99,y20170530a5463u14,sanne,estore?product_line=smartphones&product=X-phon...,Safari,4,2017-06-22 14:23:00,60.0,smartphone,,addToCart,,


#### Calculando as métricas por cada tipo de linha de produto

In [ ]:
#filtrando por product_line (linha de produto)
df.groupby(by='product_line').sum().sort_values(by='time', ascending=False)


,eventId,timestamp,nomeDesconhecido,time
product_line,,,,
,282387321341,20957867820,14,1811340.0
smartphone,1290913469370,95811510240,115,7140.0
videogames,161364183810,11981378040,82,4980.0
computers,322728367616,23961259920,47,2640.0
appliances,100852614725,7481590560,37,1560.0
headphones,161364183877,11983120680,28,1200.0
hometheater,80682091867,5988908760,14,840.0
carelectronics,40341045865,2992631580,7,360.0
cameras,40341045855,2992629720,6,300.0


## Criar um tabelão
Criamos um tabelão para juntar o dataframe original com o dataframe onde calculamos page hits (acima). 

In [ ]:
# chamamos "cs" (de clickstream) o tabelão que vai receber a mescla das duas
cs = pd.merge(df, dfPageHits, on='pageUrl', how='outer')
cs.shape

(123, 21)

Para cada ocorrência distante de page URL, ou seja, cada ação seja ela visita ou adicionar ao carrinho, haverá um cálculo de page hits diferente referente àquele endereço de URL específico. 

In [ ]:
# cria nova tabela apenas com colunas úteis

cs.columns

Index(['eventId_x', 'eventType', 'timestamp_x', 'ipaddress', 'sessionId',
       'userId', 'pageUrl', 'browser', 'nomeDesconhecido_x', 'timestampToDate',
       'time_x', 'product_line', 'product', 'action', 'feature',
       'shopping_cart', 'eventId_y', 'timestamp_y', 'nomeDesconhecido_y',
       'time_y', 'pageHits'],
      dtype='object')

Criamos um novo dataframe com todas as colunas que nos interessam.

In [ ]:
clickstream = pd.DataFrame(cs[['eventId_x', 'eventType', 'timestamp_x','timestampToDate', 'ipaddress', 'sessionId', 'userId', \
                               'pageUrl', 'browser', 'nomeDesconhecido_x', 'time_x', 'product_line', 'product', \
                               'action', 'feature', 'shopping_cart', 'pageHits']])

#### Download da tabela em excel

Faremos o download do arquivo CSV com a respectiva data e hora. 

In [ ]:
hoje = datetime.today()

In [ ]:
clickstream.to_excel('clickstream-{0}.xlsx'.format(hoje))